In [1]:
import oatpy as oat

import numpy as np
import plotly.graph_objects as go

# Generate a point cloud

In [2]:
cloud               =   oat.point_cloud.spiral_sphere(npoints=300, noise_scale=0.07, random_seed=0)

#   PLOT THE POINT CLOUD
trace               =   go.Scatter3d(x=cloud[:,0],y=cloud[:,1],z=cloud[:,2], mode="markers", marker=dict(opacity=1, size=4, color=cloud[:,2], colorscale="Aggrnyl"))
fig                 =   go.Figure(data=trace)
fig.update_layout(title=dict(text="Shere"), height=800,width=850) 
fig.show()

# Compute persistent homology

We compute persistent homology by factoring the boundary matrix.  The following cell generates a sparse distance matrix and feeds it to the persistent homology solver.  The result is a factored boundary matrix.  We will extract information from this matrix in the following cells.

In [3]:
dissimilairty_matrix    =   oat.dissimilarity.matrix_from_cloud(            
                                cloud                   =   cloud,
                                dissimilarity_max       =   0.5, # edges with length > 0.5 are excluded from the filtration
                            )

# build and factor the boundary matrix
factored                =   oat.rust.FactoredBoundaryMatrixVr( 
                                dissimilarity_matrix    =   dissimilairty_matrix,
                                homology_dimension_max  =   2,
                            )

Full details on `FactoredBoundaryMatrixVr` can be retreived with Python's `help` function.

In [4]:
help(oat.rust.FactoredBoundaryMatrixVr)

Help on class FactoredBoundaryMatrixVr in module builtins:

class FactoredBoundaryMatrixVr(object)
 |  Methods defined here:
 |  
 |  homology(self, /)
 |      Extract a barcode and a basis of cycle representatives
 |      
 |      Computes the persistent homology of the filtered clique complex (ie VR complex)
 |      with dissimilarity matrix `dismat`, over the field of rational numbers.  
 |      
 |      - Edges of weight `>= dissimilarity_max` are excluded.
 |      - Homology is computed in dimensions 0 through `homology_dimension_max`, inclusive
 |      
 |      Returns: a Pandas data frame
 |  
 |  jordan_block_indices(self, /)
 |      Returns the birth/death row/column indices of the Jordan blocks in the persistent Jordan canonical form of the filtered boundary matrix.
 |  
 |  optimize_cycle(self, /, birth_simplex, problem_type)
 |      Optimize a cycle representative
 |      
 |      Specifically, we employ the "edge loss" method to find a solution `x'` to the problem 
 |     

# Plot the persistence diagram

In [5]:
#   PLOT THE BARCODE

homology            =   factored.homology()
fig_pd              =   oat.plot.pd( homology )
fig_pd.show()

# Plot the barcode

In [6]:
fig_barcode              =   oat.plot.barcode( barcode=homology )
fig_barcode.show()

# Inspect homology and cycle representatives

The `homology` object is a data frame

In [7]:
display(homology)

dimension     birth     death    birth simplex         death simplex   
id                                                                          
0            0  0.000000  0.035873            [299]            [298, 299]  \
1            0  0.000000  0.033511            [298]            [297, 298]   
2            0  0.000000  0.045540            [297]            [295, 297]   
3            0  0.000000  0.009622            [296]            [295, 296]   
4            0  0.000000  0.016126            [295]            [294, 295]   
..         ...       ...       ...              ...                   ...   
447          1  0.203831  0.235171       [158, 159]       [158, 191, 192]   
448          1  0.201989  0.259978       [137, 170]       [136, 169, 170]   
449          1  0.048933  0.050494       [288, 290]       [288, 290, 291]   
450          2  0.428551       inf      [9, 38, 41]                  None   
451          2  0.404109  0.404886  [264, 291, 292]  [264, 289, 291, 293]   

                                  cycle representative  cycle nnz   
id                                                                  
0      simplex  filtration coefficient
0   [299]   ...          2  \
1      simplex  filtration coefficient
0   [298]   ...          2   
2      simplex  filtration coefficient
0   [297]   ...          2   
3      simplex  filtration coefficient
0   [296]   ...          2   
4      simplex  filtration coefficient
0   [295]   ...          2   
..                                                 ...        ...   
447        simplex  filtration coefficient
0  [158,...          4   
448        simplex  filtration coefficient
0  [137,...          4   
449        simplex  filtration coefficient
0  [288,...          4   
450               simplex  filtration coefficient
0...        592   
451              simplex  filtration coefficient
0 ...         26   

                                        bounding chain  bounding nnz  
id                                                                    
0          simplex  filtration coefficient
0  [298,...           2.0  
1          simplex  filtration coefficient
0  [297,...           1.0  
2          simplex  filtration coefficient
0  [295,...           2.0  
3          simplex  filtration coefficient
0  [295,...           1.0  
4          simplex  filtration coefficient
0  [294,...           1.0  
..                                                 ...           ...  
447             simplex  filtration coefficient
0  ...           2.0  
448             simplex  filtration coefficient
0  ...           2.0  
449             simplex  filtration coefficient
0  ...           2.0  
450                                               None           NaN  
451                   simplex  filtration coefficie...          22.0  

[452 rows x 9 columns]

# Inspect a cycle representative and its bounding chain

By default, terms appear in reverse filtration order (ties are broken by reverse lexicographic order)

In [8]:
homology["cycle representative"][420]

simplex  filtration coefficient
0   [130, 131]    0.230821          -1
1   [153, 154]    0.221394          -1
2   [152, 153]    0.218507          -1
3   [166, 167]    0.217590          -1
4   [132, 165]    0.215364          -1
5   [128, 161]    0.214405           1
6   [156, 157]    0.212000          -1
7   [111, 112]    0.208770          -1
8   [174, 175]    0.207021          -1
9   [114, 115]    0.204987          -1
10  [117, 118]    0.204632          -1
11  [171, 172]    0.202494          -1
12  [169, 170]    0.198902          -1
13  [160, 161]    0.198785          -1
14  [112, 113]    0.198737          -1
15  [111, 144]    0.198113           1
16  [157, 158]    0.197529          -1
17  [118, 151]    0.195270          -1
18  [116, 117]    0.194384          -1
19  [142, 175]    0.193885           1
20  [158, 191]    0.189673          -1
21  [168, 169]    0.189235          -1
22  [167, 168]    0.188447          -1
23  [159, 192]    0.187925           1
24  [142, 143]    0.183806          -1
25  [165, 166]    0.180828          -1
26  [173, 174]    0.178220          -1
27  [155, 156]    0.177620          -1
28  [170, 171]    0.176221          -1
29  [172, 173]    0.171664          -1
30  [191, 192]    0.171119          -1
31  [154, 155]    0.169257          -1
32  [129, 130]    0.169091          -1
33  [143, 144]    0.167294          -1
34  [113, 114]    0.160309          -1
35  [115, 116]    0.158581          -1
36  [128, 129]    0.156394          -1
37  [151, 152]    0.148597          -1
38  [159, 160]    0.146037          -1
39  [131, 132]    0.137389          -1

In [9]:
homology["bounding chain"][420]

simplex  filtration coefficient
0    [253, 284, 285]    0.419060           1
1    [249, 253, 284]    0.419060          -1
2    [254, 285, 287]    0.412998           1
3    [254, 255, 287]    0.412998          -1
4    [249, 279, 284]    0.403305           1
..               ...         ...         ...
289  [294, 295, 298]    0.077675          -1
290  [283, 284, 286]    0.076447          -1
291  [291, 292, 293]    0.073131           1
292  [282, 283, 284]    0.072960           1
293  [292, 293, 294]    0.058021          -1

[294 rows x 3 columns]

# Plot a representative

In [10]:
#   FIND THE LONGEST BAR IN DIMENSION 1

def lifetime(p):
     """
     gets the lifetime of a feature; returns -infinity for features of dimension != 1
     """
     if homology["dimension"][p]!= 1:
          return -np.Inf
     else:
          return homology["death"][p] - homology["birth"][p]

max( 
     range(homology.shape[0]),  # number of rows in the data frame
     key    =   lifetime,
)

420

In [11]:
#   PLOT

edges               =   homology["cycle representative"][420]["simplex"].tolist() # the cycle
triangles           =   homology["bounding chain"][420]["simplex"].tolist() # the chain that bounds the cycle
coo                 =   cloud # coo stands for coordinate oracle

traces_edge         =   [ oat.plot.edge__trace3d( edge, coo  ) for edge in edges ]
trace_triangle      =   oat.plot.triangles__trace3d( triangles, coo ) 
trace_cloud         =   go.Scatter3d(x=cloud[:,0],y=cloud[:,1],z=cloud[:,2], mode="markers", marker = dict(opacity=0.5, size=3, color=cloud[:,1], colorscale="Aggrnyl"))

trace_cloud.update(showlegend=True, opacity=0.5, name="Point cloud")
trace_triangle.update(showlegend=True, legendgroup="triangles", opacity=0.5, name="Bounding chain", color="black")
for trace_number, trace in enumerate( traces_edge ):
    showlegend      =   trace_number == 0
    trace.update(showlegend=showlegend, legendgroup="edges", opacity=0.5, name="Initial cycle", line=dict(color="red", width=10))

fig = go.Figure(data= [trace_cloud] + traces_edge + [trace_triangle] )
fig.update_layout(title=dict(text="Cycle representative and bounding chain"))
fig.update_layout(height=800,width=850) 
fig.show()

Suggestion: consider using `triangle__trace3d` instead of `triangles__trace3d`, as the former often gives higher quality graphics.
